In [1]:
import dv_processing as dv

In [19]:
import dv_processing as dv
import os

import dv_processing as dv
import os
import argparse

def cut_aedat4_file(input_file, output_file, keep_percentage=30):
    """
    Cut the beginning of an AEDAT4 file and keep only the last percentage
    
    Args:
        input_file (str): Path to input AEDAT4 file
        output_file (str): Path to output AEDAT4 file
        keep_percentage (int): Percentage to keep from the end (default: 30%)
    """
    print(f"Opening input file: {input_file}")
    reader = dv.io.MonoCameraRecording(input_file)
    
    # Get resolution from the reader
    resolution = (reader.getEventResolution()[0], reader.getEventResolution()[1])
    print(f"Detected resolution: {resolution}")
    
    # Create a writer configuration and output file
    config = dv.io.MonoCameraWriter.EventOnlyConfig(os.path.basename(input_file).split('.')[0], resolution)
    writer = dv.io.MonoCameraWriter(output_file, config)
    
    # First, count the total number of event batches to determine the cutoff point
    total_events = 0
    all_event_batches = []
    
    print("Counting events in the file...")
    while reader.isRunning():
        events = reader.getNextEventBatch()
        if events is not None:
            total_events += len(events)
            all_event_batches.append(events)
        else:
            break
    
    print(f"Total events: {total_events}")
    
    # Calculate how many events to skip (70% of total)
    events_to_skip = int(total_events * (100 - keep_percentage) / 100)
    print(f"Events to skip (keeping last {keep_percentage}%): {events_to_skip}")
    
    # Skip events and then write the rest
    events_processed = 0
    for events in all_event_batches:
        if events_processed + len(events) <= events_to_skip:
            # Skip this entire batch
            events_processed += len(events)
        else:
            # We need to start writing from this batch
            # First, handle the partial first batch if needed
            if events_processed < events_to_skip:
                # Calculate how many events to skip in this batch
                skip_in_batch = events_to_skip - events_processed
                
                # Create a new event store with just the events we want to keep
                partial_events = dv.EventStore()
                for i in range(skip_in_batch, len(events)):
                    partial_events.push_back(events[i])
                
                writer.writeEvents(partial_events)
            else:
                # Write the full batch
                writer.writeEvents(events)
            
            events_processed += len(events)
    
    events_kept = total_events - events_to_skip
    print(f"Written {events_kept} events to {output_file}")
    print(f"Successfully kept the last {keep_percentage}% of the file")


cut_aedat4_file(r"D:\UET\SNN\Visual-Tactile-Sensing-with-Event-Cameras-using-Spiking-Neural-Networks\data\raw_data\raw1.aedat4", "test1.aedat4", keep_percentage=25)

Opening input file: D:\UET\SNN\Visual-Tactile-Sensing-with-Event-Cameras-using-Spiking-Neural-Networks\data\raw_data\raw1.aedat4
Detected resolution: (240, 180)
Counting events in the file...
Total events: 957795
Events to skip (keeping last 25%): 718346
Written 239449 events to test1.aedat4
Successfully kept the last 25% of the file


In [13]:
reader = dv.io.MonoCameraRecording(r"D:\UET\SNN\Visual-Tactile-Sensing-with-Event-Cameras-using-Spiking-Neural-Networks\data\raw_data\raw1.aedat4")
    
resolution = reader.getEventResolution()

In [15]:
resolution[1]

180